<a href="https://colab.research.google.com/github/Kaua1217/aula_Karlla/blob/main/atividade_hostipal_14_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import heapq
from typing import Dict, List, Optional, Tuple, Any

# --- 1. CONSTANTES E Mapeamentos ---
COR_PRIORIDADE: Dict[str, int] = {
    "vermelho": 4,  # Prioridade 4 (mais alta)
    "laranja": 3,
    "amarelo": 2,
    "azul": 1       # Prioridade 1 (mais baixa)
}

TEMPO_CONSULTA_MINUTOS: int = 20
ESPECIALIDADES: List[str] = [
    "Clínica Médica", "Pediatria", "Ortopedia",
    "Cardiologia", "Ginecologia/Obstetrícia", "Otorrinolaringologia"
]

# --- 2. FUNÇÕES DE UTILIDADE DE TEMPO ---

def converter_para_minutos(hora_hh_mm: str) -> int:
    """Converte 'HH:MM' para minutos desde 00:00."""
    try:
        h, m = map(int, hora_hh_mm.split(':'))
        return h * 60 + m
    except ValueError:
        raise ValueError(f"Formato de hora inválido: {hora_hh_mm}. Use HH:MM.")

def converter_para_hh_mm(minutos: int) -> str:
    """Converte minutos totais para o formato 'HH:MM'."""
    h = minutos // 60
    m = minutos % 60
    return f"{h:02d}:{m:02d}"

# --- 3. CLASSE PACIENTE (Data Structure) ---

class Paciente:
    """Armazena os dados do paciente para triagem."""
    def __init__(self, nome: str, idade: int, cor_urgencia: str, especialidade: str, hora_chegada_str: str):
        if cor_urgencia.lower() not in COR_PRIORIDADE:
            raise ValueError(f"Cor de urgência inválida: {cor_urgencia}")

        self.nome: str = nome
        self.idade: int = idade
        self.cor_urgencia: str = cor_urgencia.lower()
        self.especialidade: str = especialidade
        self.hora_chegada_minutos: int = converter_para_minutos(hora_chegada_str)
        self.prioridade_valor: int = COR_PRIORIDADE[self.cor_urgencia]

    def __repr__(self) -> str:
        # Mantemos o __repr__ mas garantimos que não é usado para comparação pelo heap
        return f"Paciente(Nome={self.nome}, Urgencia={self.cor_urgencia}, Chegada={converter_para_hh_mm(self.hora_chegada_minutos)})"

# --- 4. CLASSE PRINCIPAL DE GERENCIAMENTO (Core Logic) ---

class TriagemHospitalar:
    """Gerencia as filas de prioridade por especialidade."""
    def __init__(self):
        # Dicionário: Especialidade -> Fila de Prioridade (Lista Heap)
        self.filas_especialidade: Dict[str, List[Tuple[Tuple[int, int, int], Paciente]]] = {
            esp: [] for esp in ESPECIALIDADES
        }

        # Dicionário: Especialidade -> Minuto de Fim da Última Consulta
        self.fim_ultima_consulta: Dict[str, int] = {}

        # CONTADOR GLOBAL PARA DESEMPATE ÚNICO NO HEAP (CORREÇÃO DO ERRO)
        self.ordem_chegada_global: int = 0

    def adicionar_paciente(self, paciente: Paciente) -> None:
        """
        Adiciona um paciente à fila de prioridade da especialidade.
        A chave de prioridade é: (-prioridade, hora_chegada_minutos, ordem_chegada_global).
        O terceiro elemento (ordem_chegada_global) garante desempate único.
        """
        esp = paciente.especialidade

        if esp not in self.filas_especialidade:
            print(f"AVISO: Adicionando nova especialidade ({esp}).")
            self.filas_especialidade[esp] = []

        # Incrementa o contador global de ordem de chegada
        self.ordem_chegada_global += 1

        # Chave de Prioridade: (-Prioridade, Hora de Chegada em Minutos, Ordem Global Única)
        # 1. -Prioridade: Maior Urgência primeiro.
        # 2. Hora de Chegada: FIFO entre mesma prioridade (se o minuto for diferente).
        # 3. Ordem Global: Desempate final para evitar erro de comparação do objeto Paciente.
        chave_prioridade = (
            -paciente.prioridade_valor,
            paciente.hora_chegada_minutos,
            self.ordem_chegada_global
        )

        # Insere a tupla (chave_prioridade, objeto_paciente) no heap
        heapq.heappush(self.filas_especialidade[esp], (chave_prioridade, paciente))

        # Inicializa o tempo de fim da última consulta se for o primeiro paciente
        if esp not in self.fim_ultima_consulta:
            self.fim_ultima_consulta[esp] = paciente.hora_chegada_minutos

    def proximo(self, especialidade: str) -> Tuple[Paciente, str, str]:
        """
        Remove o paciente de maior prioridade e atualiza a agenda do médico.
        """
        fila = self.filas_especialidade.get(especialidade)
        if not fila:
            raise IndexError(f"A fila de {especialidade} está vazia ou a especialidade é inválida.")

        # Remove e retorna o elemento de menor tupla (maior prioridade)
        (chave, proximo_paciente) = heapq.heappop(fila)

        # Simulação da Agenda: O tempo em que o médico fica livre para o paciente anterior
        tempo_medico_livre = self.fim_ultima_consulta.get(especialidade, proximo_paciente.hora_chegada_minutos)

        # Previsão de Início: Max(Hora de Chegada, Médico Livre)
        previsao_inicio = max(proximo_paciente.hora_chegada_minutos, tempo_medico_livre)

        # Fim da Consulta = Início + Duração Fixa
        previsao_fim = previsao_inicio + TEMPO_CONSULTA_MINUTOS

        # Atualiza o tempo de disponibilidade do médico para o próximo paciente
        self.fim_ultima_consulta[especialidade] = previsao_fim

        return (
            proximo_paciente,
            converter_para_hh_mm(previsao_inicio),
            converter_para_hh_mm(previsao_fim)
        )

    def previsao_atendimento(self, especialidade: str) -> List[Dict[str, Any]]:
        """
        Simula a agenda do médico para todos os pacientes na fila,
        mantendo o estado atual da fila inalterado.
        """
        fila_original = self.filas_especialidade.get(especialidade)
        if not fila_original:
            return []

        # 1. Copia a fila (lista) e transforma em heap para ter a ordem correta
        # Agora funciona corretamente devido à chave de 3 elementos no heap
        fila_copia = list(fila_original)
        heapq.heapify(fila_copia)

        # 2. Inicializa o tempo de 'médico livre'
        tempo_medico_livre = self.fim_ultima_consulta.get(especialidade, 0)

        lista_previsao = []

        while fila_copia:
            # Remove o próximo paciente de maior prioridade da CÓPIA
            (chave, paciente) = heapq.heappop(fila_copia)

            # Previsão de Início = Max(Hora de Chegada do Paciente, Hora em que o Médico Fica Livre)
            previsao_inicio = max(paciente.hora_chegada_minutos, tempo_medico_livre)

            previsao_fim = previsao_inicio + TEMPO_CONSULTA_MINUTOS

            # Tempo de espera = Tempo que o paciente esperou APÓS a chegada até o início
            tempo_espera = previsao_inicio - paciente.hora_chegada_minutos

            lista_previsao.append({
                "nome": paciente.nome,
                "urgencia": paciente.cor_urgencia,
                "chegada": converter_para_hh_mm(paciente.hora_chegada_minutos),
                "inicio_previsto": converter_para_hh_mm(previsao_inicio),
                "fim_previsto": converter_para_hh_mm(previsao_fim),
                "tempo_espera_min": tempo_espera
            })

            # O médico estará livre após o fim desta consulta
            tempo_medico_livre = previsao_fim

        return lista_previsao

    def listar_fila(self, especialidade: str) -> List[str]:
        """Mostra os pacientes na ordem prevista de atendimento."""
        previsoes = self.previsao_atendimento(especialidade)
        return [
            f"{p['nome']} ({p['urgencia'].capitalize()}, Início: {p['inicio_previsto']})"
            for p in previsoes
        ]

    def tempo_medio_espera(self, especialidade: str, hora_atual_str: str) -> float:
        """
        Calcula o tempo médio de espera (após a chegada) dos pacientes
        ainda na fila.
        """
        # A hora_atual_str é irrelevante para o cálculo, pois a média
        # considera a espera potencial (Hora Prevista - Hora Chegada)
        # dos pacientes AINDA NA FILA.

        previsoes = self.previsao_atendimento(especialidade)
        if not previsoes:
            return 0.0

        total_espera_min = sum(p['tempo_espera_min'] for p in previsoes)

        # Média = Soma dos tempos de espera / Número de pacientes na fila
        return total_espera_min / len(previsoes)


# --- 5. EXECUÇÃO E TESTE ---

# Inicializa o sistema
hospital = TriagemHospitalar()

# Dados de Exemplo (Mínimo para Testar)
print("--- 1. CADASTRO DE PACIENTES ---")
hospital.adicionar_paciente(Paciente("Maria Silva", 34, "laranja", "Ortopedia", "08:05")) # Ordem 1
hospital.adicionar_paciente(Paciente("João Souza", 62, "amarelo", "Clínica Médica", "08:07")) # Ordem 2
hospital.adicionar_paciente(Paciente("Ana Lima", 5, "vermelho", "Pediatria", "08:09")) # Ordem 3
hospital.adicionar_paciente(Paciente("Carla Dias", 41, "azul", "Ortopedia", "08:10")) # Ordem 4
hospital.adicionar_paciente(Paciente("Pedro Neri", 58, "laranja", "Cardiologia", "08:12")) # Ordem 5
hospital.adicionar_paciente(Paciente("Rui Campos", 29, "amarelo", "Ortopedia", "08:14")) # Ordem 6
hospital.adicionar_paciente(Paciente("Lucas Velo", 25, "amarelo", "Ortopedia", "08:14")) # Ordem 7 (Mesma cor/hora de Rui)
print("Pacientes cadastrados.\n")

# TESTE A: Previsão e Ordem de Atendimento
print("--- 2. ORDEM E PREVISÃO DE ATENDIMENTO (Ortopedia) ---")
print("Fila ordenada de Ortopedia:")
print(hospital.listar_fila("Ortopedia"))
previsao_ortopedia = hospital.previsao_atendimento("Ortopedia")

for p in previsao_ortopedia:
    print(f"  > {p['nome']} ({p['urgencia'].capitalize()}): Chegada {p['chegada']}, Início {p['inicio_previsto']}, Fim {p['fim_previsto']}, Espera {p['tempo_espera_min']} min")

print("\n--- 3. TEMPO MÉDIO DE ESPERA (Hora Atual: 08:30) ---")
hora_atual = "08:30"
tme_ortopedia = hospital.tempo_medio_espera("Ortopedia", hora_atual)
print(f"Tempo Médio de Espera (Ortopedia): {tme_ortopedia:.2f} minutos")

# TESTE B: Simulação de Chamada (Atualiza Agenda)
print("\n--- 4. CHAMANDO PRÓXIMOS PACIENTES ---")
try:
    p_ped, inicio_ped, fim_ped = hospital.proximo("Pediatria")
    print(f"PRÓXIMO DE PEDIATRIA: {p_ped.nome} - Início: {inicio_ped}, Fim: {fim_ped}")

    p_ort, inicio_ort, fim_ort = hospital.proximo("Ortopedia")
    print(f"PRÓXIMO DE ORTOPEDIA: {p_ort.nome} - Início: {inicio_ort}, Fim: {fim_ort}")

    # Próximo de ortopedia é Rui (amarelo, 08:14) ou Lucas (amarelo, 08:14).
    # Rui (ordem 6) deve vir antes de Lucas (ordem 7).
    print(f"\nHorário de Fim da Última Consulta (Ortopedia) após chamada de Maria: {converter_para_hh_mm(hospital.fim_ultima_consulta['Ortopedia'])}")

    # Início será às 08:25 (fim da Maria)
    p_ort2, inicio_ort2, fim_ort2 = hospital.proximo("Ortopedia")
    print(f"PRÓXIMO DE ORTOPEDIA (2): {p_ort2.nome} - Início: {inicio_ort2}, Fim: {fim_ort2}")

    # Próximo de ortopedia é Lucas (amarelo, 08:14). Início será às 08:45 (fim do Rui)
    p_ort3, inicio_ort3, fim_ort3 = hospital.proximo("Ortopedia")
    print(f"PRÓXIMO DE ORTOPEDIA (3): {p_ort3.nome} - Início: {inicio_ort3}, Fim: {fim_ort3}")

except IndexError as e:
    print(f"Erro: {e}")

print("\n--- 5. FILA DE ORTOPEDIA RESTANTE ---")
print("Fila ordenada de Ortopedia (Restante):")
print(hospital.listar_fila("Ortopedia"))

# TESTE C: Caso de Borda (Fila Vazia)
try:
    hospital.proximo("Pediatria")
except IndexError as e:
    print(f"\nTeste Fila Vazia: {e}")

--- 1. CADASTRO DE PACIENTES ---
Pacientes cadastrados.

--- 2. ORDEM E PREVISÃO DE ATENDIMENTO (Ortopedia) ---
Fila ordenada de Ortopedia:
['Maria Silva (Laranja, Início: 08:05)', 'Rui Campos (Amarelo, Início: 08:25)', 'Lucas Velo (Amarelo, Início: 08:45)', 'Carla Dias (Azul, Início: 09:05)']
  > Maria Silva (Laranja): Chegada 08:05, Início 08:05, Fim 08:25, Espera 0 min
  > Rui Campos (Amarelo): Chegada 08:14, Início 08:25, Fim 08:45, Espera 11 min
  > Lucas Velo (Amarelo): Chegada 08:14, Início 08:45, Fim 09:05, Espera 31 min
  > Carla Dias (Azul): Chegada 08:10, Início 09:05, Fim 09:25, Espera 55 min

--- 3. TEMPO MÉDIO DE ESPERA (Hora Atual: 08:30) ---
Tempo Médio de Espera (Ortopedia): 24.25 minutos

--- 4. CHAMANDO PRÓXIMOS PACIENTES ---
PRÓXIMO DE PEDIATRIA: Ana Lima - Início: 08:09, Fim: 08:29
PRÓXIMO DE ORTOPEDIA: Maria Silva - Início: 08:05, Fim: 08:25

Horário de Fim da Última Consulta (Ortopedia) após chamada de Maria: 08:25
PRÓXIMO DE ORTOPEDIA (2): Rui Campos - Início: 0